In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import scipy.io
import cv2
import glob
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import torch.utils.data
%matplotlib inline



In [2]:

df=pd.read_csv('/home/dev/Letöltések/mpii_dataset.csv')



In [3]:
from sklearn.model_selection import train_test_split
df.drop(['Category','Activity'],1)
df = df.sample(n = len(df)).reset_index(drop=True)

/tmp/ipykernel_63484/934581437.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['Category','Activity'],1)


In [4]:
df.head()
from torch.utils.data import Dataset


In [5]:
transferdata_df = df.iloc[- 1400:, 0:]

test_df = pd.DataFrame()
test_df= test_df.append(transferdata_df)
train_df =df.drop(transferdata_df.index)
test_df = test_df.reset_index(drop=True)
test_df.head()

/tmp/ipykernel_63484/1936806666.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df= test_df.append(transferdata_df)


,Unnamed: 0,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,...,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0,0,066668565.jpg,748,549,664,471,738,366,724,366,...,235,697,242,683,325,650,290,1.884420,"running, marathon",running
1,0,046484980.jpg,1025,947,770,634,1037,584,1220,581,...,265,1260,243,1245,569,1146,733,7.170018,"sitting, arts and crafts, carving wood, weavi...",miscellaneous
2,0,053304329.jpg,879,714,864,642,849,545,880,546,...,413,846,405,809,370,817,337,1.512762,"standing quietly, standing in a line",inactivity quiet/light
3,0,070662987.jpg,-1,-1,393,391,399,248,350,269,...,97,246,147,236,217,218,211,2.681564,fishing from river bank,fishing and hunting
4,0,090408171.jpg,400,425,400,357,392,283,426,282,...,198,438,202,448,232,452,261,1.358705,"ballet, modern, or jazz",dancing


In [37]:
img_loc=("/run/media/dev/Windows_7_Ultimate_32_Bit/Images/images/")
#imgArray = np.array([])
#files = glob.glob(img_loc+'*.jpg')
imgcomb=[]


images=[]       
trainimglabels=train_df.loc[:,'r ankle_X':'Scale']
trainfilenames=train_df['NAME']

testimglabels=test_df.loc[:,'r ankle_X':'Scale']
testfilenames=test_df['NAME']

class Imagedataset(Dataset):
    def __init__(self,fnames,labels,transform=None):
        self.fnames=fnames
        self.labels=labels
        self.transform=transform
    
    def __len__(self):
        return len(self.fnames)
    
    def NormaliseData(self,img,idx):
        #print(self.labels.head())
        height, width = img.shape
        #print(self.labels.head())
        Xcoord=[]
        Ycoord=[]
        x=0
        for i in range(0,31,2):
            self.labels.iloc[idx,i]=(self.labels.iloc[idx,i]*(256/width))
            Xcoord.append(self.labels.iloc[idx,i])
            self.labels.iloc[idx,i+1]=(self.labels.iloc[idx,i+1]*(256/height))
            Ycoord.append(self.labels.iloc[idx,i+1])
            x=x+1
        X_min=np.min(Xcoord)
        Y_min=np.min(Ycoord)
        X_max=np.max(Xcoord)
        Y_max=np.max(Ycoord)
        x=0
        for i in range(0,31,2):
            self.labels.iloc[idx, i] = (self.labels.iloc[idx, i] - X_min) / (X_max - X_min)
            self.labels.iloc[idx, i+1] = (self.labels.iloc[idx, i+1] - Y_min) / (Y_max - Y_min)
            x=x+1
        #print(self.labels.head())
        normImg=cv2.resize(img,(256,256))
        return normImg


    def __getitem__ (self, idx):
        Rimg=cv2.imread(img_loc+self.fnames[idx])
        Gimg=cv2.cvtColor(Rimg,cv2.COLOR_BGR2GRAY)
        Gimg=self.NormaliseData(Gimg,idx)
        #for i in range(0,31,2):
         #   y=self.labels.iloc[idx,i+1]
          #  x=self.labels.iloc[idx,i]
           # cv2.circle(Gimg,(int(x),int(y)),5,(0,255,0),-1)
        #cv2.imshow("gay", Gimg)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        Gimg = Gimg.astype(np.float32) / 255.
        if self.transform:
            Gimg = self.transform(Gimg)
       
        helper=self.labels.iloc[idx, :-1]
       
        target=helper.values
        target=target.reshape(-1,2)
     
        return (Gimg,target)

    
train_dataset= Imagedataset(trainfilenames,trainimglabels)
test_dataset= Imagedataset(testfilenames,testimglabels)

train_loader=DataLoader(train_dataset,batch_size=15, shuffle=True, num_workers=4)
test_loader=DataLoader(test_dataset,batch_size=15, shuffle=True)




In [22]:
data, target= train_dataset.__getitem__(1)
asdf=pd.DataFrame(data)
print(data.shape)


(256, 256)


In [23]:
train_dataset.fnames.iloc[4]
import itertools

In [24]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1)
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(256*16*16,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,128)
        self.fc4 = nn.Linear(128,16*2)

    def forward(self, X):
        X = self.conv1(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv2(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv3(X)
        X = F.relu(X)
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = self.conv4(X)
        X = F.relu(X)
     
        X = F.max_pool2d(X,kernel_size=2,stride=2)
        X = torch.flatten(X,1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = F.relu(X)
        X = self.fc4(X)
        X=X.view(-1,16,2)
        output = F.log_softmax(X, dim=1)
        return output
    
torch.manual_seed(101)
model = ConvolutionalNetwork().to(dev)
print(dev)

cpu


In [25]:

criterion = nn.BCEWithLogitsLoss() #try diceloss, MultiLabelSoftMarginLoss
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [40]:
def train(model, dev, train_loader, optimizer, epoch):
  model.train()
  
  for batch_idx in range(0,len(train_loader),15):
    #print("dik")
    #start_idx = batch_idx
    #end_idx = min(len(train_loader), batch_idx + 20)
    batch=list(itertools.islice(train_loader,15))
    for data, target in batch:
      #print("coc")
      data=data.unsqueeze(1)
      #print(data.shape ," MIAGECI ",target.shape)
      data, target = data.to(dev), target.to(dev)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      loss.backward()
      optimizer.step()
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
      epoch, batch_idx * len(data), len(train_loader.dataset),
      100. * batch_idx / len(train_loader), loss.item()))   
    
    
        

In [38]:

train_dataset.__getitem__(0)


(array([[0.52156866, 0.53333336, 0.53333336, ..., 0.7019608 , 0.69803923,
         0.69803923],
        [0.52156866, 0.53333336, 0.53333336, ..., 0.7019608 , 0.69411767,
         0.6901961 ],
        [0.52156866, 0.53333336, 0.53333336, ..., 0.7019608 , 0.6901961 ,
         0.6862745 ],
        ...,
        [0.22352941, 0.18039216, 0.17254902, ..., 0.42352942, 0.42745098,
         0.45490196],
        [0.24705882, 0.18039216, 0.17254902, ..., 0.43137255, 0.43529412,
         0.45490196],
        [0.22745098, 0.18431373, 0.17254902, ..., 0.43137255, 0.43529412,
         0.45490196]], dtype=float32),
 array([[0.33516628, 0.92309837],
        [0.33516628, 0.69775871],
        [0.33516628, 0.45990018],
        [0.85444469, 0.45990018],
        [0.91738753, 0.72279645],
        [0.92918931, 1.        ],
        [0.59480548, 0.45990018],
        [0.42564661, 0.10937182],
        [0.48937191, 0.12574633],
        [0.        , 0.        ],
        [0.39024126, 0.38299855],
        [0.14633777,

In [28]:
torch.cuda.empty_cache()

In [41]:
for epoch in range(1, 6):
    train(model, dev, train_loader, optimizer, epoch)

Train Epoch: 1 [0/15972 (0%)]	Loss: 1804979.401
Train Epoch: 1 [0/15972 (0%)]	Loss: 1733464.101
Train Epoch: 1 [0/15972 (0%)]	Loss: 1533093.711
Train Epoch: 1 [0/15972 (0%)]	Loss: 1355121.247
Train Epoch: 1 [0/15972 (0%)]	Loss: 1257885.471
Train Epoch: 1 [0/15972 (0%)]	Loss: 1134656.349
Train Epoch: 1 [0/15972 (0%)]	Loss: 1057270.157
Train Epoch: 1 [0/15972 (0%)]	Loss: 1016185.965
Train Epoch: 1 [0/15972 (0%)]	Loss: 891320.151
Train Epoch: 1 [0/15972 (0%)]	Loss: 784170.314
Train Epoch: 1 [0/15972 (0%)]	Loss: 698220.308
Train Epoch: 1 [0/15972 (0%)]	Loss: 674644.896
Train Epoch: 1 [0/15972 (0%)]	Loss: 580601.354
Train Epoch: 1 [0/15972 (0%)]	Loss: 547313.799
Train Epoch: 1 [0/15972 (0%)]	Loss: 547803.029


KeyboardInterrupt: 